# Si bien dice EDA, es mas un Scrapper. La informacion se analiza con Looker

Librerias

In [131]:
import pandas as pd

Leyendo las 2 primeras tablas de la wiki del ZZZ

In [132]:
agentsdfUrl = pd.read_html('https://zenless-zone-zero.fandom.com/wiki/Agent')
agentsdf = agentsdfUrl[1]
agentsUpcomingdf = agentsdfUrl[2]
agentsdf.head()

,Icon,Name,Rank,Attribute,Fighting Style,Faction,Model Type
0,NaN,Anby,NaN,Electric,Slash,Cunning Hares,Female
1,NaN,Anton,NaN,Electric,Pierce,Belobog Heavy Industries,Male
2,NaN,Ben,NaN,Fire,Strike,Belobog Heavy Industries,Male
3,NaN,Billy,NaN,Physical,Pierce,Cunning Hares,Male
4,NaN,Corin,NaN,Physical,Slash,Victoria Housekeeping,Female


Eliminando columnas que no son utiles ya que contienen imagenes

In [133]:
agentsdf.drop(columns=['Icon', 'Rank'], inplace=True)
agentsUpcomingdf.drop(columns=['Icon', 'Rank'], inplace=True)

In [134]:
agentsdf.describe()

,Name,Attribute,Fighting Style,Faction,Model Type
count,14,14,14,14,14
unique,14,5,4,5,2
top,Anby,Electric,Slash,Cunning Hares,Female
freq,1,4,6,4,10


Agregando columna para ver el status del pj dentro del juego

In [135]:
agentsdf['Status'] = 'Playable'
agentsUpcomingdf['Status'] = 'Upcoming'

Uniendo los df

In [136]:
agents_df = pd.concat([agentsdf, agentsUpcomingdf], ignore_index=True)
agents_df

,Name,Attribute,Fighting Style,Faction,Model Type,Status
0,Anby,Electric,Slash,Cunning Hares,Female,Playable
1,Anton,Electric,Pierce,Belobog Heavy Industries,Male,Playable
2,Ben,Fire,Strike,Belobog Heavy Industries,Male,Playable
3,Billy,Physical,Pierce,Cunning Hares,Male,Playable
4,Corin,Physical,Slash,Victoria Housekeeping,Female,Playable
5,Ellen,Ice,Slash,Victoria Housekeeping,Female,Playable
6,Grace,Electric,Pierce,Belobog Heavy Industries,Female,Playable
7,Koleda,Fire,Strike,Belobog Heavy Industries,Female,Playable
8,Lycaon,Ice,Strike,Victoria Housekeeping,Male,Playable
9,Nekomata,Physical,Slash,Cunning Hares,Female,Playable


# Aca se busca mas informacion para agregar al dataframe.
En primer lugar se crean dos arrays, uno vacio y otro con las columnas deseadas(se va a utilizar mas adelante debido a errores que se ocasionaban).
Se buclea una URL segun el nombre de los personajes del dataframe anterior(se agrega un except ya que hay nombres con espacios).
Se intenta leer la tabla en indice 2 de la url y se printea, luego se realizan transformaciones necesarias como dropear columnas innecesarias, renombrar y transpose. Se resetea index y se utiliza la primer fila como headers, tambien se agrega la columna Name para poder unir los dataframe mas adelante. Si no se encuentra la tabla necesaria para poder hacer las transformaciones, se trae el arreglo creado anteriormente y se da valor de 0 a todos los campos. Una vez que se bucleo sobre cada personaje, ambos dataframe se unifican

In [137]:
dfs = []
desired_columns = ['HP', 'ATK', 'DEF', 'Impact', 'CRIT Rate', 'Crit DMG', 'PEN Ratio', 'PEN', 'Energy Regen', 'Attribute Mastery', 'Name', 'Energy Recovery']



for name in agents_df['Name']:
    url_name = name.replace(' ', '_')
    url = 'https://zenless-zone-zero.fandom.com/wiki/' + url_name
   
    try:
        df = pd.read_html(url)[2]
        print(url)

        df.drop(columns=['2', '3', '4', '5', '6', '7', '8', '9', '10'], inplace=True)
        df.rename(columns={'Unnamed: 0': 'Stats', '1': 'Base_Stats'}, inplace=True)
        df2 = df.transpose()
        df2.reset_index(inplace=True)
        df2.drop(columns=['index'], inplace=True)
        df2.columns = df2.iloc[0]
        df2 = df2[1:]
        df2['Name'] = name

        dfs.append(df2)

    except:
        print(name + ' not found')
        dummy_df = pd.DataFrame({col: 0 for col in desired_columns}, index=[0])
        dummy_df['Name'] = name
        dfs.append(dummy_df)


final_df = pd.concat(dfs, ignore_index=True)
final_df

https://zenless-zone-zero.fandom.com/wiki/Anby
https://zenless-zone-zero.fandom.com/wiki/Anton
https://zenless-zone-zero.fandom.com/wiki/Ben
https://zenless-zone-zero.fandom.com/wiki/Billy
https://zenless-zone-zero.fandom.com/wiki/Corin
https://zenless-zone-zero.fandom.com/wiki/Ellen
Ellen not found
https://zenless-zone-zero.fandom.com/wiki/Grace
Grace not found
https://zenless-zone-zero.fandom.com/wiki/Koleda
https://zenless-zone-zero.fandom.com/wiki/Lycaon
https://zenless-zone-zero.fandom.com/wiki/Nekomata
https://zenless-zone-zero.fandom.com/wiki/Nicole
https://zenless-zone-zero.fandom.com/wiki/Rina
Rina not found
https://zenless-zone-zero.fandom.com/wiki/Soldier_11
https://zenless-zone-zero.fandom.com/wiki/Soukaku
https://zenless-zone-zero.fandom.com/wiki/Miyabi


,HP,ATK,DEF,Impact,CRIT Rate,Crit DMG,PEN Ratio,PEN,Energy Regen,Attribute Mastery,Name,Energy Recovery
0,603,95,49,115,5%,50%,0%,0,1.2,88,Anby,NaN
1,462,101,48,110,5%,50%,0%,0,NaN,NaN,Anton,1.8
2,668,94,56,103,5%,50%,0%,0,1.2,80,Ben,NaN
3,555,113,49,91,5%,50%,0%,0,2.4,100,Billy,NaN
4,484,106,48,120,5%,50%,0%,0,NaN,NaN,Corin,1.6
5,0,0,0,0,0,0,0,0,0,0,Ellen,0
6,0,0,0,0,0,0,0,0,0,0,Grace,0
7,431,100,45,100,5%,50%,0%,0,NaN,NaN,Koleda,2
8,473,96,50,110,5%,50%,0%,0,NaN,NaN,Lycaon,1.8
9,512,111,55,100,5%,50%,0%,0,NaN,NaN,Nekomata,2


Unificacion de los dataframe segun Name

In [138]:
agents_df = pd.merge(agents_df, final_df, on='Name')
agents_df.head()

,Name,Attribute,Fighting Style,Faction,Model Type,Status,HP,ATK,DEF,Impact,CRIT Rate,Crit DMG,PEN Ratio,PEN,Energy Regen,Attribute Mastery,Energy Recovery
0,Anby,Electric,Slash,Cunning Hares,Female,Playable,603,95,49,115,5%,50%,0%,0,1.2,88,NaN
1,Anton,Electric,Pierce,Belobog Heavy Industries,Male,Playable,462,101,48,110,5%,50%,0%,0,NaN,NaN,1.8
2,Ben,Fire,Strike,Belobog Heavy Industries,Male,Playable,668,94,56,103,5%,50%,0%,0,1.2,80,NaN
3,Billy,Physical,Pierce,Cunning Hares,Male,Playable,555,113,49,91,5%,50%,0%,0,2.4,100,NaN
4,Corin,Physical,Slash,Victoria Housekeeping,Female,Playable,484,106,48,120,5%,50%,0%,0,NaN,NaN,1.6


Se vizualiza que Energy Regen y Energy Recovery son las mismas columnas con diferentes nombres, ademas de la relacion en la cual si existe valor en una columna en la otra es NaN. Por lo que se genera una nueva columna uniendo los valores de ambas y se procede a eliminar las 2 anteriores

In [139]:
agents_df['Energy Regeneration'] = agents_df['Energy Regen'].fillna(agents_df['Energy Recovery'])
agents_df.drop(columns=['Energy Regen', 'Energy Recovery'], inplace=True)
agents_df.head()

,Name,Attribute,Fighting Style,Faction,Model Type,Status,HP,ATK,DEF,Impact,CRIT Rate,Crit DMG,PEN Ratio,PEN,Attribute Mastery,Energy Regeneration
0,Anby,Electric,Slash,Cunning Hares,Female,Playable,603,95,49,115,5%,50%,0%,0,88,1.2
1,Anton,Electric,Pierce,Belobog Heavy Industries,Male,Playable,462,101,48,110,5%,50%,0%,0,NaN,1.8
2,Ben,Fire,Strike,Belobog Heavy Industries,Male,Playable,668,94,56,103,5%,50%,0%,0,80,1.2
3,Billy,Physical,Pierce,Cunning Hares,Male,Playable,555,113,49,91,5%,50%,0%,0,100,2.4
4,Corin,Physical,Slash,Victoria Housekeeping,Female,Playable,484,106,48,120,5%,50%,0%,0,NaN,1.6


Se completan los valores NaN de Mastery con 0

In [140]:
agents_df['Attribute Mastery'].fillna(0, inplace=True)
agents_df.head()

,Name,Attribute,Fighting Style,Faction,Model Type,Status,HP,ATK,DEF,Impact,CRIT Rate,Crit DMG,PEN Ratio,PEN,Attribute Mastery,Energy Regeneration
0,Anby,Electric,Slash,Cunning Hares,Female,Playable,603,95,49,115,5%,50%,0%,0,88,1.2
1,Anton,Electric,Pierce,Belobog Heavy Industries,Male,Playable,462,101,48,110,5%,50%,0%,0,0,1.8
2,Ben,Fire,Strike,Belobog Heavy Industries,Male,Playable,668,94,56,103,5%,50%,0%,0,80,1.2
3,Billy,Physical,Pierce,Cunning Hares,Male,Playable,555,113,49,91,5%,50%,0%,0,100,2.4
4,Corin,Physical,Slash,Victoria Housekeeping,Female,Playable,484,106,48,120,5%,50%,0%,0,0,1.6


Se guarda el DF para poder importar la informacion a Looker y graficar

In [141]:
agents_df.to_csv('agentsZzz.csv', index=False)